In [ ]:
import openai
import time

api_key = ''
openai_client = openai.OpenAI(
    api_key=api_key,
    base_url="https://openrouter.ai/api/v1"
)
def run_openai_query(prompt, history, completion_tokens, temp, model, openai_client):
    response = None
    while True:
        try:
            messages = history + [{"role": "system", "content": "Write a chapter following the prompt."}, {"role": "user", "content": prompt}]
            
            response = openai_client.chat.completions.create(
                    model=model,
                    temperature=temp,                
                    max_tokens=completion_tokens,                
                    messages=messages
            )
            content = response.choices[0].message.content
                
            if content:
                return content.strip()
            else:
                print(response)
                print('Error: message is empty')
                time.sleep(5)

        except Exception as e:
            print(response)
            print("Request failed.")
            print(e)
            time.sleep(5)

    return None

In [10]:
import os
import json
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

with open("data/site/processing/site_scenes_with_prompts_fiction.json", 'r') as f:
    scenes = json.load(f)

outfile = 'data/site/processing/site_scenes_with_rewrites_fiction.json'
rewrites = {}
if os.path.exists(outfile):
    with open(outfile, 'r') as f:
        rewrites = json.load(f)

def process_scene(scene):
    # Perform the OpenAI query
    result = run_openai_query(scene['prompt'], [], 2000, 0.7, "google/gemma-2-9b-it", openai_client)
    
    if result:
        scene['rejected'] = result
    return scene

for fn, v in tqdm(scenes.items()):
    if fn in rewrites:
        print('skipping', fn)
        continue
    if not v:
        continue
    
    this_scenes = []
    # Use ThreadPoolExecutor for multithreading
    with ThreadPoolExecutor(max_workers=len(v)) as executor:
        # Submit all scenes to the executor
        futures = {executor.submit(process_scene, scene): scene for scene in v}

        # Process completed futures
        for future in as_completed(futures):
            scene = futures[future]
            try:
                processed_scene = future.result()
                if processed_scene:
                    this_scenes.append(processed_scene)
            except Exception as e:
                print(f"Error processing scene {scene['prompt']}: {e}")

    rewrites[fn] = this_scenes

    # Save results to file
    with open(outfile, 'w') as f:
        json.dump(rewrites, f, indent=2)
    


 33%|███▎      | 21/64 [05:22<12:24, 17.32s/it]

None
Request failed.
Expecting value: line 749 column 1 (char 4114)


100%|██████████| 64/64 [16:39<00:00, 15.61s/it]
